In [55]:
#PublicKey Generation
def key_gen(m,n,P,Q,l,e,Po,Qo,E):
    R = m*P+n*Q
    Ec = E
    Pi = Po
    Qi = Qo
    for i in range(0,1):
        K = (l^(e-1-i))*R
        ### sage
        #phi = EllipticCurveIsogeny(Ec, K)
        #Ec = phi.codomain()
        #R = phi(R)
        #Pi = phi(Pi)
        #Qi = phi(Qi)
        #### kamil
        Ec, Pi, Qi = Short_Weierstrass_Velu(E, K, Pi, Qi)
    return Ec, Pi, Qi

#PK for initiator
def key_gen_init(mA,nA):
    return key_gen(mA,nA,PA,QA,lA,eA,PB,QB,E)

#PK for responder
def key_gen_resp(mB,nB):
    return key_gen(mB,nB,PB,QB,lB,eB,PA,QA,E)

########################################################################

#Key Derivation from Own Private and Other Public information
#Input: own private key, own order, other EC, other aux points
def key_der(m,n,l,e,E,P,Q):
    R = m*P+n*Q
    Ec = E
    for i in range(0,1):
        K = (l^(e-1-i))*R
        #sage
        #phi = EllipticCurveIsogeny(Ec, K)
        #Ec = phi.codomain()
        #R = phi(R)
        #kamil
        Ec, _, _ = Short_Weierstrass_Velu(Ec, K, P, Q)
    key = Short_Weierstrass_j_invariant(Ec)
    return key

#Key Derivation by initiator
def key_der_init(m,n,E,P,Q):
    return key_der(m,n,lA,eA,E,P,Q)

#Key Derivation by responder
def key_der_resp(m,n,E,P,Q):
    return key_der(m,n,lB,eB,E,P,Q)

########################################################################

#Testing and test vectors
def test_KA(mA,nA,mB,nB,E):
    In = key_gen_init(mA,nA)
    Re = key_gen_resp(mB,nB)
    kA = key_der_init(mA,nA,Re[0],Re[1],Re[2])
    kB = key_der_resp(mB,nB,In[0],In[1],In[2])
    if kA == kB:
        print("Shared key:", kA)
    else:
        return "ERROR in KA"

def Elliptic_Point_Order(P):
    i = 1
    while P * i != 0:
        P = P * i
        i += 1
    return i
    
def Short_Weierstrass_j_invariant(E):
    a4 = E.a4()
    a6 = E.a6()
    b4 = 2 * a4
    b6 = 4 * a6
    b8 = a4**2 * -1
    c4 = -24 * b4
    c6 = -216 * b6
    d = -8 * b4**3 -27 * b6**2
    j = c4**3 / d
    return j
    
def Short_Weierstrass_Velu(E, gen, P, Q):
    n = Elliptic_Point_Order(gen)
    i = 1
    g2 = []
    rp = []
    rm = []
    d = E.a4()
    e = E.a6()
    while i != n: #partitioning points
        pnt = i * gen
        if pnt * 2 == 0:
            g2.append(pnt)
        else:
            if pnt not in rp and -pnt not in rp:
                rp.append(pnt)
                rm.append(-pnt)
        i += 1
    S = g2 + rp
    v = 0
    w = 0
    dPx = 0
    dQx = 0
    dPy = 0
    dQy = 0
    for point in S:
        fq = 3 * point[0]^2 + d
        gq = -2 * point[1]
        vq = 0
        if point * 2 == 0:
            vq = fq
        else:
            vq = 2 * fq
        uq = gq ^ 2
        v += vq
        w += (uq + point[0] * vq)
        ############ isogeny part
        dPx += vq / (P[0] - point[0]) + (uq / (P[0] - point[0])**2) 
        dQx += vq / (Q[0] - point[0]) + (uq / (Q[0] - point[0])**2)
        dPy += (uq * 2*P[1] / (P[0] - point[0])^3) + (vq * (P[1] - point[1]) / (P[0] - point[0])^2) - (fq * gq / (P[0] - point[0])^2)
        dQy += (uq * 2*Q[1] / (Q[0] - point[0])^3) + (vq * (Q[1] - point[1]) / (Q[0] - point[0])^2) - (fq * gq / (Q[0] - point[0])^2) 
        ############ point image part
    phi_P = (P[0] + dPx, P[1] - dPy)
    phi_Q = (Q[0] + dQx, Q[1] - dQy)
    #print(dPx)
    d_E = d - 5 * v
    e_E = e - 7 * w
    Ec = EllipticCurve([d_E, e_E])
    return Ec, Ec(phi_P), Ec(phi_Q)

def point_image(P, vq, uq, fq, gq, point):
    dPx = vq / (P[0] - point[0]) + uq / (P[0] - point[0])^2 
    dPy = (uq * 2*P[1] / (P[0] - point[0])^3) + (vq * (P[1] - point[1]) / (P[0] - point[0])^2) - (fq * gq / (P[0] - point[0])^2)
    return dPx, dPy

#Parameters

#Curve
lA = 2
lB = 3
eA = 6
eB = 1
f = 1
p = (lA^eA)*(lB^eB)*f - 1
F.<z> = GF(p^2)
E = EllipticCurve([F(1), F(0)]) #ie. y^2=x^3+x
pt = E(46, 113)

########################################################################

#Points
#PAx = F(2374093068336250774107936421407893885897*z + 2524646701852396349308425328218203569693)
#PAy = F(1944869260414574206229153243510104781725*z + 1309099413211767078055232768460483417201)
#PA = E(PAx,PAy)
#QA = E(F(-PAx),F(z*PAy))

#PBx = F(1556716033657530876728525059284431761206*z + 1747407329595165241335131647929866065215)
#PBy = F(3456956202852028835529419995475915388483*z + 1975912874247458572654720717155755005566)
#PB = E(PBx,PBy)
#QB = E(F(-PBx),F(z*PBy))

PA=E(F(189*z+177), F(174*z+27))
QA=E(F(132*z+170), F(151*z+56))

PB=E(F(31*z+80), F(64*z+179))
QB=E(F(145), F(56*z+163))

#Example, random integer coefficients, not both divisible by lA|lB respectively
mA = 27
nA = 69

mB = 17
nB = 32

test_KA(mA,nA,mB,nB,E)

Shared key: 11*z + 113
